# SD-WEBUI

## Install packages

Check GPU + Cuda version

In [ ]:
!nvidia-smi

Install xformers to speed up inference

In [ ]:
!pip install xformers huggingface-hub

Downgrade torchvision

In [ ]:
!pip install torchvision==0.15.2+cu117 --index-url https://download.pytorch.org/whl/cu117

Check if GPU is working

In [ ]:
import torch
torch.cuda.is_available()

## Declare var + functions

In [ ]:
import os
import git
import shutil
from pathlib import Path

# Home path (only works on Linux)
# https://stackoverflow.com/a/58988310
home_dir = Path(os.readlink('/proc/%s/cwd' % os.environ['JPY_PARENT_PID']))

# SD paths
sd_dir = home_dir / 'stable-diffusion-webui'
sd_model_dir = sd_dir / 'models'
sd_extensions_dir = sd_dir / 'extensions'

In [ ]:
# Clone a repo
def clone_repo(git_url, target_dir):
    
    # Extract repo name and create path
    repo_name = git_url.split('/')[-1]

    # Create path
    repo_dir = target_dir / repo_name

    # Clone if does not exist
    if not repo_dir.exists():
        git.Repo.clone_from(git_url, repo_dir)

**⚠️** Warning: only run the next cell if you want to delete the existing copy of sd-webui **⚠️**

In [ ]:
answer = input("-> Are you sure you want to delete 'stable-diffusion-webui' [y | n]: ")
if answer == 'y':
    if sd_dir.exists():
        shutil.rmtree(sd_dir)
        print('Folder deleted')
    else:
        print('Folder not found')
else:
    print('Skipping')

## Install SD REPO

Clone and cd into repo

In [ ]:
clone_repo('https://github.com/AUTOMATIC1111/stable-diffusion-webui', home_dir)

#### 1. Manually modify the file `stable-diffusion/webui.sh`
Change `use_venv=1` to `use_venv=0`
#### 2. Manually modify the file `stable-diffusion/webui-user.sh`
Dont forget to uncomment the line!! (delete the `#`)  
change `#export COMMANDLINE_ARGS=""` to  `export COMMANDLINE_ARGS="--xformers --share"`

## Install extensions

In [ ]:
# Use urls without the .git
git_urls = {
    'https://github.com/zanllp/sd-webui-infinite-image-browsing',
    'https://github.com/Mikubill/sd-webui-controlnet'
}

# Iterate through git repos and clone them
for url in git_urls:
    clone_repo(url, sd_extensions_dir)

## Download models

In [ ]:
import huggingface_hub

# SD models
local_dir = sd_model_dir / 'Stable-diffusion'

# SD 1.5
repo_id='runwayml/stable-diffusion-v1-5'
filename = 'v1-5-pruned-emaonly.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# SDXL
repo_id='stabilityai/stable-diffusion-xl-base-1.0'
# Base model
filename = 'sd_xl_base_1.0.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)
# VAE
filename = 'sd_xl_base_1.0_0.9vae.safetensors'
huggingface_hub.hf_hub_download(
    repo_id=repo_id,
    filename=filename,
    local_dir=local_dir
)

# ControlNet (There's a LOT here, hopefully we have enough disk space 🤞)
local_dir = sd_model_dir / 'ControlNet'

# ControlNet 1.1 
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/ControlNet-v1-1',
    allow_patterns='*.pth',
    local_dir=local_dir
)
# ControlNet XL
huggingface_hub.snapshot_download(
    repo_id='lllyasviel/sd_control_collection',
    local_dir=local_dir
)

## Run UI

In [ ]:
# Change dir
os.chdir(sd_dir)

In [ ]:
!./webui.sh